# Suggest locations for opening a Restaurant chain in Toronto
### Manish Minni

In [1]:
!conda install -c conda-forge folium=0.5.0 --yes
#!pip install ibm-cos-sdk
!pip install geopy

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  20.26 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.26 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  39.52 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  48.10 MB/s
Requirement not upgraded as not directly required: geopy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages


In [2]:
import ibm_boto3
from botocore.client import Config
import json

from pandas.io.json import json_normalize
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from geopy.distance import distance

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

import matplotlib.cm as cm
import matplotlib.colors as colors
#from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
#import matplotlib.cm as cm
#import numpy as np
from folium.plugins import HeatMap
#from folium.features import DivIcon

#from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import sklearn.utils
from sklearn.preprocessing import StandardScaler


In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
# The code was removed by Watson Studio for sharing.

In [5]:

def load_string(fileobject):
    '''Load the file contents into a Python string'''
    text = fileobject.read().decode("utf-8") 
    return text

def load_dict(fileobject):
    '''Load the file contents into a Python dict'''
    text = fileobject.read().decode("utf-8") 
    dictformat = json.loads(text)
    return dictformat

In [6]:
'''Retrieve file from Cloud Object Storage'''
fileobject = cos.get_object(Bucket=credentials_c1['BUCKET'], Key='NEIGHBORHOODS_WGS84.geojson')['Body']
geojson_c1 = load_string(fileobject)

In [7]:
address = 'Toronto'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude_c1 = location.latitude
longitude_c1 = location.longitude
#print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [8]:
increment=0.01
max_km=15
limits_list=[]
i = -1
while True:
    i+=1
    j = -1
    while True:
        j+=1
        dist = distance([latitude_c1, longitude_c1],[latitude_c1 + i * increment, longitude_c1 + j * increment])
        #print(i,j,dist.km)
        if dist.km > max_km:
            break
    limits_list.append([i, [latitude_c1 + i * increment, longitude_c1 - j * increment], [latitude_c1 + i * increment, longitude_c1 + j * increment]])    
    if i > 0:
        limits_list.append([-i, [latitude_c1 - i * increment, longitude_c1 - j * increment], [latitude_c1 - i * increment, longitude_c1 + j * increment]])    
    if j == 0:
        break

#print(latitude_c1, longitude_c1)

limits = pd.DataFrame(limits_list).sort_values(0, ascending=False)
limits.columns=['y', 'location_min', 'location_max']
limits


,y,location_min,location_max
27,14,"[43.793963, -79.387207]","[43.793963, -79.387207]"
25,13,"[43.783963, -79.447207]","[43.783963, -79.327207]"
23,12,"[43.773962999999995, -79.477207]","[43.773962999999995, -79.297207]"
21,11,"[43.763963, -79.497207]","[43.763963, -79.277207]"
19,10,"[43.753963, -79.517207]","[43.753963, -79.25720700000001]"
17,9,"[43.743963, -79.527207]","[43.743963, -79.247207]"
15,8,"[43.733962999999996, -79.53720700000001]","[43.733962999999996, -79.237207]"
13,7,"[43.723963, -79.547207]","[43.723963, -79.227207]"
11,6,"[43.713963, -79.557207]","[43.713963, -79.217207]"
9,5,"[43.703962999999995, -79.56720700000001]","[43.703962999999995, -79.207207]"


In [9]:
#CLIENT_ID initialized above
#CLIENT_SECRET initialized above
#VERSION = '20180605' initialized above
LIMIT = 50
    
def get_venues(names, latitudes, longitudes, radius):

    i = 0
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        i += 1
        if i > 10:
            break
        get_near_venues( name, lat, lng, radius=radius, venues_list=venues_list)

    found_venues = pd.DataFrame(venues_list)
    found_venues.columns = ['near_name', 
                  'near_latitude', 
                  'near_longitude', 
                  'venue_id', 
                  'venue_name', 
                  'venue_latitude', 
                  'venue_longitude', 
                  'category_id',
                  'category_name'         
                            ]
    
    return(found_venues)

def get_near_venues(name, lat, lng, radius, venues_list=[]):
    
    more = True
    offset = 0
    print(name)
    
    
    while more:
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&offset={}&time=any&day=any'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            offset)
       
        # make the GET request
        #print(url)
        
        r = 0
        results = None
        while r < 5: 
            if r > 0:
                print ('***retrying {}***'.format(r))
            try:
                r+= 1
                results_full = requests.get(url).json()
                results = results_full["response"]['groups'][0]['items']
                break
            except Exception:
                print('***Exception***')
                pass

        if results is None:
            print('***Foursquare Error*************************')
            continue
            
        if len(results) == 0:
            print('ok')
            more = False
            continue
           
        offset += len(results)         
        
        # return only relevant information for each nearby venue
        venues_list.extend([(
            name, 
            lat, 
            lng, 
            #borough,
            v['venue']['id'], 
            v['venue']['name'], 
            #v['venue']['location']['city'] if 'city' in v['venue']['location'] else '',  
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['id'],
            v['venue']['categories'][0]['name'],
            #v['venue']['categories'][0]['icon']['prefix'] if 'icon' in v['venue']['categories'][0] else ''
        ) for v in results])
        
        #print(name, len(venues_list), len(results))
           

In [10]:
interval = 0.01
extra = interval / 10 # this will be used to to ensure that np.arange does not miss the last point
locations_list = []
for loc_min, loc_max in zip(limits.location_min, limits.location_max):
    #print(loc_min, loc_max, loc_max[1] + extra)
    lat = loc_min[0]
    for lon in np.arange(loc_min[1], loc_max[1] + extra, interval):
        locations_list.append([str(round(lat,3)) + ',' + str(round(lon,3)), lat, lon])

locations = pd.DataFrame(locations_list)
locations.columns=['name','latitude','longitude']
#locations

In [11]:
found_venues = get_venues(names=locations['name'], latitudes=locations['latitude'], longitudes=locations['longitude'], radius=1000 ) 

found_venues.shape

43.794,-79.387
ok
43.784,-79.447
ok
43.784,-79.437
ok
43.784,-79.427
ok
43.784,-79.417
ok
43.784,-79.407
ok
43.784,-79.397
ok
43.784,-79.387
ok
43.784,-79.377
ok
43.784,-79.367
ok


(234, 9)

In [12]:
   
url = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)

r = 0
results = None
while r < 5: 
    if r > 0:
        print ('***retrying {}***'.format(r))
    try:
        r+= 1
        results = requests.get(url).json()
        break
    except Exception:
        print('***Exception***')
        pass

if results is None:
    print('***Foursquare Error*************************')
    


def get_categories( cats, parent='|', cat_list=[] ):
    for cat in cats:
        name = cat['shortName'].strip()
        cat_list.append([cat['id'], name, parent + name + '|'])
        get_categories( cat['categories'], parent + name + '|', cat_list )

categories_list=[]
get_categories( results['response']['categories'], cat_list=categories_list)
                         
categories=pd.DataFrame(categories_list, columns=['category_id','category_name','qualified_category'])
#print(categories.shape)

food_categories = categories[categories['qualified_category'].str.startswith('|Food|')]

#print(food_categories.shape)
#food_categories.sort_values('category_name')


In [13]:
venues = found_venues.groupby(['venue_id'])['venue_name', 'venue_latitude', 'venue_longitude', 'category_id', 'category_name'].max().reset_index()
#print(len(venues.venue_id.unique()))

#print(venues.shape)
venues = venues.merge(categories, how='left', on='category_id', suffixes=('_venue',''))
#print(venues.shape)
venues.rename(columns={'venue_latitude':'latitude','venue_longitude':'longitude'}, inplace=True)
venues.drop(columns=['category_name_venue'], inplace=True)
#venues.head(1)

In [14]:
food_venues = venues[venues['qualified_category'].str.startswith('|Food|')]
food_venues['latitude_rnd'] = round(food_venues['latitude'], 3)
#print(food_venues)
food_venues.shape

sklearn.utils.check_random_state(1000)
clus_dataset = food_venues[['latitude','longitude']]
clus_dataset = np.nan_to_num(clus_dataset)
clus_dataset = StandardScaler().fit_transform(clus_dataset)

# Compute DBSCAN

eps = 0.31
while True:
    eps -= 0.01
    db = DBSCAN(eps=eps, min_samples=30).fit(clus_dataset)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_
    food_venues['clus_num']=labels
    clus_cnt = food_venues[['clus_num','venue_id']][food_venues['clus_num'] >= 0].groupby(['clus_num']).count() 
    clus_cnt.rename(columns={'venue_id': 'venue_cnt'}, inplace=True)
    if clus_cnt['venue_cnt'].max() <= 250 or eps < 0.05:
        break
        


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
clus_cnt

,venue_cnt
clus_num,
0,37


In [16]:
clus_info = food_venues[['clus_num','latitude','longitude']][food_venues['clus_num'] >= 0].groupby(['clus_num']).median() 
clus_info = clus_info.merge(clus_cnt, how='inner', left_index=True, right_index=True)

clus_info['polygon'] = None

for clus_num in food_venues['clus_num'].unique():
    if clus_num < 0:
        continue
    clus_polygon = []
    clus_venues = food_venues[food_venues['clus_num'] == clus_num]
    [clus_polygon.append([ lat, clus_venues[clus_venues['latitude_rnd'] == lat]['longitude'].min()]) for lat in sorted(clus_venues['latitude_rnd'].unique())]
    [clus_polygon.append([ lat, clus_venues[clus_venues['latitude_rnd'] == lat]['longitude'].max()]) for lat in sorted(clus_venues['latitude_rnd'].unique(), reverse=True)]
    clus_info.at[clus_num, 'polygon'] = clus_polygon
    
#print(clus_info.dtypes)


In [17]:
for ndx, row in clus_info.iterrows():
    print(ndx, row['latitude'])

0 43.778292380641865


In [18]:


venue_map = folium.Map(location=[latitude_c1, longitude_c1], zoom_start=12)
venue_map.choropleth(geo_data=geojson_c1,
    #data = df_pop,
    #olumns=['Geographic code','Population, 2016'],
    #key_on='feature.properties.CFSAUID',
    #fill_color='YlOrRd',                    
    fill_opacity=0, 
    line_opacity=0.25,
    legend_name='')   

color_list = list(dict(colors.BASE_COLORS, **colors.CSS4_COLORS))

#colors = plt.get_cmap('jet')(np.linspace(0.0, 1.0, clusterNum))
#print(colors)
# instantiate a feature group for the markers in the dataframe

venue_locations = [[row['latitude'], row['longitude']] for ndx, row in food_venues.iterrows()]
heat_map = HeatMap( venue_locations,
                     min_opacity=0.05,
                     radius=6, blur=14,
                     max_zoom=13   )
    
marker_map = folium.map.FeatureGroup()

# loop through the neighborhoods and add each to the markers feature group
for ndx, row in clus_info.iterrows():
    marker_map.add_child(
        folium.features.Circle(
            [row['latitude'], row['longitude']],
            radius=200, 
            color='black',
            weight = 1.5,
            fill=False,
            fill_color='black',
            fill_opacity=1
        )
    )
    
    marker_map.add_child(
        folium.features.PolygonMarker(
            locations=row['polygon'],
            color='purple',
            weight= 1,
            fill_color='red',
            fill_opacity=0
        )
    )


    #text = str(row['venue_cnt'])
    #marker_map.add_child(
    #    folium.map.Marker(
    #        [row['latitude'], row['longitude']],
    #        icon=DivIcon(
    #            icon_size=(150,36),
    #            icon_anchor=(0,0),
    #            html='<div style="font-size: 16pt">%s</div>' % text,
    #        )
    #    )
    #)
    
venue_map.add_child(heat_map)
venue_map.add_child(marker_map)



In [19]:
print(found_venues.shape)
print(venues.shape)
print(food_venues.shape)
clus_info

(234, 9)
(148, 7)
(101, 9)


,latitude,longitude,venue_cnt,polygon
clus_num,,,,
0,43.778292,-79.415099,37,"[[43.776, -79.4146476965], [43.777, -79.414938..."


In [20]:
clus_cnt.sum()

venue_cnt    37
dtype: int64

In [21]:
food_venues.clus_num.value_counts()

-1    64
 0    37
Name: clus_num, dtype: int64

In [22]:
food_venues.clus_num.value_counts().sum()

101

In [23]:
labels

array([-1,  0, -1, -1,  0, -1, -1, -1,  0, -1, -1, -1, -1, -1,  0,  0, -1,
        0, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1,
       -1, -1,  0, -1, -1, -1, -1,  0, -1,  0, -1,  0, -1, -1, -1,  0,  0,
       -1, -1, -1, -1,  0,  0, -1, -1, -1, -1,  0,  0,  0,  0, -1,  0,  0,
       -1, -1,  0,  0,  0, -1,  0,  0, -1,  0, -1,  0, -1,  0, -1, -1, -1,
       -1,  0,  0,  0, -1,  0, -1, -1, -1, -1,  0, -1,  0,  0, -1, -1])

In [24]:
list(set(labels))

[0, -1]

In [25]:
l = pd.DataFrame(labels)